<a href="https://colab.research.google.com/github/goodaycoder/LLDprogression/blob/master/LLDProgressionClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchio
!pip install einops
!pip install monai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 172 kB 31.0 MB/s 
     |████████████████████████████████| 52.8 MB 250 kB/s 
     |████████████████████████████████| 237 kB 17.0 MB/s 
     |████████████████████████████████| 51 kB 4.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 515 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 39.9 MB/s 


In [ ]:
import os, sys, argparse
import enum
import time
import datetime
import random
import multiprocessing
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from scipy import stats
from IPython import display
from tqdm.notebook import tqdm

import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, roc_auc_score, matthews_corrcoef
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.preprocessing import normalize

from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

import torchio as tio
import monai
from google.colab import drive
#drive.mount('/content/drive')

from typing import Sequence, Union

import re
import torch
import torch.nn as nn
from torchsummary import summary

from monai.networks.blocks.patchembedding import PatchEmbeddingBlock
from monai.networks.blocks.transformerblock import TransformerBlock
from monai.networks.nets import resnet18, resnet34, resnet50, ViT
from einops.layers.torch import Rearrange

In [ ]:
#cur_dir = os.getcwd()
#pkg_rootdir = os.path.dirname(os.path.dirname(cur_dir))
#if pkg_rootdir not in sys.path:
#    sys.path.append(pkg_rootdir)
LLDdir = '/content/drive/MyDrive/Colab Notebooks/LLD_Progression'
sys.path.append(LLDdir)
#from Model_HRL import HRL34_v1, HRL34_v2

#Model Define

In [ ]:
from typing import Sequence, Type, Union
from functools import partial
from typing import Any, Callable, List, Optional, Tuple, Type, Union

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LayerNorm

#from monai.networks.layers import trunc_normal_
from monai.networks.layers.factories import Conv, Norm, Pool
from monai.networks.layers.utils import get_pool_layer
from monai.utils import ensure_tuple_rep, optional_import
from monai.utils.module import look_up_option


from monai.networks.blocks.patchembedding import PatchEmbeddingBlock
from monai.networks.blocks.transformerblock import TransformerBlock
from monai.networks.nets.resnet import ResNetBlock, ResNetBottleneck, ResNet
class ResNetBackbone(nn.Module):
    def __init__(
        self,
        block: Union[Type[Union[ResNetBlock, ResNetBottleneck]], str],
        layers: List[int],
        block_inplanes: List[int],
        spatial_dims: int = 3,
        n_input_channels: int = 1,
        conv1_t_size: Union[Tuple[int], int] = 7,
        conv1_t_stride: Union[Tuple[int], int] = 1,
        no_max_pool: bool = False,
        shortcut_type: str = "B",
        widen_factor: float = 1.0,
        num_classes: int = 3,
        feed_forward: bool = True,
    ) -> None:

        super().__init__()

        if isinstance(block, str):
            if block == "basic":
                block = ResNetBlock
            elif block == "bottleneck":
                block = ResNetBottleneck
            else:
                raise ValueError("Unknown block '%s', use basic or bottleneck" % block)

        conv_type: Type[Union[nn.Conv1d, nn.Conv2d, nn.Conv3d]] = Conv[Conv.CONV, spatial_dims]
        norm_type: Type[Union[nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d]] = Norm[Norm.BATCH, spatial_dims]
        pool_type: Type[Union[nn.MaxPool1d, nn.MaxPool2d, nn.MaxPool3d]] = Pool[Pool.MAX, spatial_dims]
        avgp_type: Type[Union[nn.AdaptiveAvgPool1d, nn.AdaptiveAvgPool2d, nn.AdaptiveAvgPool3d]] = Pool[
            Pool.ADAPTIVEAVG, spatial_dims
        ]

        #block_avgpool = get_avgpool()
        block_inplanes = [int(x * widen_factor) for x in block_inplanes]

        self.in_planes = block_inplanes[0]
        self.no_max_pool = no_max_pool

        conv1_kernel_size = ensure_tuple_rep(conv1_t_size, spatial_dims)
        conv1_stride = ensure_tuple_rep(conv1_t_stride, spatial_dims)

        self.conv1 = conv_type(
            n_input_channels,
            self.in_planes,
            kernel_size=conv1_kernel_size,  # type: ignore
            stride=conv1_stride,  # type: ignore
            padding=tuple(k // 2 for k in conv1_kernel_size),  # type: ignore
            bias=False,
        )
        self.bn1 = norm_type(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = pool_type(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, block_inplanes[0], layers[0], spatial_dims, shortcut_type)
        self.layer2 = self._make_layer(block, block_inplanes[1], layers[1], spatial_dims, shortcut_type, stride=2)
        self.layer3 = self._make_layer(block, block_inplanes[2], layers[2], spatial_dims, shortcut_type, stride=2)
        self.layer4 = self._make_layer(block, block_inplanes[3], layers[3], spatial_dims, shortcut_type, stride=2)
        #self.avgpool = avgp_type(block_avgpool[spatial_dims])
        #self.fc = nn.Linear(block_inplanes[3] * block.expansion, num_classes) if feed_forward else None

        for m in self.modules():
            if isinstance(m, conv_type):
                nn.init.kaiming_normal_(torch.as_tensor(m.weight), mode="fan_out", nonlinearity="relu")
            elif isinstance(m, norm_type):
                nn.init.constant_(torch.as_tensor(m.weight), 1)
                nn.init.constant_(torch.as_tensor(m.bias), 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(torch.as_tensor(m.bias), 0)

    def _downsample_basic_block(self, x: torch.Tensor, planes: int, stride: int, spatial_dims: int = 3) -> torch.Tensor:
        out: torch.Tensor = get_pool_layer(("avg", {"kernel_size": 1, "stride": stride}), spatial_dims=spatial_dims)(x)
        zero_pads = torch.zeros(out.size(0), planes - out.size(1), *out.shape[2:], dtype=out.dtype, device=out.device)
        out = torch.cat([out.data, zero_pads], dim=1)
        return out

    def _make_layer(
        self,
        block: Type[Union[ResNetBlock, ResNetBottleneck]],
        planes: int,
        blocks: int,
        spatial_dims: int,
        shortcut_type: str,
        stride: int = 1,
    ) -> nn.Sequential:

        conv_type: Callable = Conv[Conv.CONV, spatial_dims]
        norm_type: Callable = Norm[Norm.BATCH, spatial_dims]

        downsample: Union[nn.Module, partial, None] = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            if look_up_option(shortcut_type, {"A", "B"}) == "A":
                downsample = partial(
                    self._downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride,
                    spatial_dims=spatial_dims,
                )
            else:
                downsample = nn.Sequential(
                    conv_type(self.in_planes, planes * block.expansion, kernel_size=1, stride=stride),
                    norm_type(planes * block.expansion),
                )

        layers = [
            block(
                in_planes=self.in_planes, planes=planes, spatial_dims=spatial_dims, stride=stride, downsample=downsample
            )
        ]

        self.in_planes = planes * block.expansion
        for _i in range(1, blocks):
            layers.append(block(self.in_planes, planes, spatial_dims=spatial_dims))

        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x


    

class HRL34_v1(nn.Module):
    def __init__(
        self,
        in_channels: int,
        img_size: Union[Sequence[int], int],
        patch_size: Union[Sequence[int], int],
        hidden_size: int = 128,
        mlp_dim: int = 512,
        num_layers: int = 1,
        num_heads: int = 16,
        pool = 'mean',
        pos_embed: str = "conv",
        classification: bool = True,
        num_classes: int = 2,
        dropout_rate: float = 0.0,
        spatial_dims: int = 3,
        post_activation="Tanh",
    ) -> None:
        """
        Args:
            in_channels: dimension of input channels.
            img_size: dimension of input image.
            patch_size: dimension of patch size.
            hidden_size: dimension of hidden layer.
            mlp_dim: dimension of feedforward layer.
            num_layers: number of transformer blocks.
            num_heads: number of attention heads.
            pos_embed: position embedding layer type.
            classification: bool argument to determine if classification is used.
            num_classes: number of classes if classification is used.
            dropout_rate: faction of the input units to drop.
            spatial_dims: number of spatial dimensions.
            post_activation: add a final acivation function to the classification head when `classification` is True.
                Default to "Tanh" for `nn.Tanh()`. Set to other values to remove this function.

        """

        super().__init__()

        if not (0 <= dropout_rate <= 1):
            raise ValueError("dropout_rate should be between 0 and 1.")

        if hidden_size % num_heads != 0:
            raise ValueError("hidden_size should be divisible by num_heads.")

        self.ResNetBack = ResNetBackbone(ResNetBlock, [3, 4, 6, 3], [64, 128, 256, 512],conv1_t_stride=2)#resnet34
        #self.ResNetBack = ResNetBackbone(ResNetBlock, [3, 4, 6, 3], [64, 128, 256, 512])#resnet34

        self.pool = pool
        self.classification = classification
        self.in_channels = in_channels
        self.hidden_size = hidden_size
        self.patch_embedding1 = PatchEmbeddingBlock(
            in_channels=in_channels,
            img_size=img_size,
            patch_size=patch_size,
            hidden_size=hidden_size,
            num_heads=num_heads,
            pos_embed=pos_embed,
            dropout_rate=dropout_rate,
            spatial_dims=spatial_dims,
        )
        self.blocks1 = nn.ModuleList(
            [TransformerBlock(hidden_size, mlp_dim, num_heads, dropout_rate) for i in range(num_layers)]
        )
        self.norm1 = nn.LayerNorm(hidden_size)
        if self.classification:
            self.cls_token1 = nn.Parameter(torch.zeros(1, 1, hidden_size))
            if post_activation == "Tanh":
                self.classification_head = nn.Sequential(nn.Linear(hidden_size, num_classes), nn.Tanh())
            else:
                self.classification_head = nn.Linear(hidden_size, num_classes)  # type: ignore


    def forward(self, x1):
        x1 = self.ResNetBack(x1)
        #x1 = x1.reshape(x1.shape[0],x1.shape[1],-1)# batchsize*512*flatten_size
        if self.in_channels == 1:
            x1 = x1.reshape(x1.shape[0],1,-1,x1.shape[3],x1.shape[4])
        x1 = self.patch_embedding1(x1)
        x1 = x1.reshape(x1.shape[0],-1,self.hidden_size)
        
        if hasattr(self, "cls_token"):
            cls_token1 = self.cls_token1.expand(x1.shape[0], -1, -1)
            x1 = torch.cat((cls_token1, x1), dim=1)
            #cls_token2 = self.cls_token2.expand(x2.shape[0], -1, -1)
            #x2 = torch.cat((cls_token2, x2), dim=1)
        hidden_states_out1 = []
        for blk in self.blocks1:
            x1 = blk(x1)
            hidden_states_out1.append(x1)
        x1 = self.norm1(x1)

        
        if hasattr(self, "classification_head"):
            x1 = x1.mean(dim=1) if self.pool == 'mean' else x1[:, 0]
            #x2 = x2.mean(dim=1) if self.pool == 'mean' else x2[:, 0]
            #x = self.classification_head(torch.cat((x1, x2), 1))
            x1 = self.classification_head(x1)
        return x1, hidden_states_out1

class HRL34_v2(nn.Module):
    def __init__(
        self,
        in_channels: int,
        img_size: Union[Sequence[int], int],
        patch_size: Union[Sequence[int], int],
        hidden_size1: int = 128,
        hidden_size2: int = 128,
        mlp_dim: int = 512,
        num_layers: int = 1,
        num_heads: int = 16,
        pool = 'mean',
        pos_embed: str = "conv",
        classification: bool = True,
        num_classes: int = 2,
        dropout_rate: float = 0.0,
        spatial_dims: int = 3,
        post_activation="Tanh",
    ) -> None:
        """
        Args:
            in_channels: dimension of input channels.
            img_size: dimension of input image.
            patch_size: dimension of patch size.
            hidden_size: dimension of hidden layer.
            mlp_dim: dimension of feedforward layer.
            num_layers: number of transformer blocks.
            num_heads: number of attention heads.
            pos_embed: position embedding layer type.
            classification: bool argument to determine if classification is used.
            num_classes: number of classes if classification is used.
            dropout_rate: faction of the input units to drop.
            spatial_dims: number of spatial dimensions.
            post_activation: add a final acivation function to the classification head when `classification` is True.
                Default to "Tanh" for `nn.Tanh()`. Set to other values to remove this function.

        """

        super().__init__()

        if not (0 <= dropout_rate <= 1):
            raise ValueError("dropout_rate should be between 0 and 1.")

        if hidden_size1 % num_heads != 0:
            raise ValueError("hidden_size should be divisible by num_heads.")

        self.ResNetBack = ResNetBackbone(ResNetBlock, [3, 4, 6, 3], [64, 128, 256, 512],conv1_t_stride=2)#resnet34
        #self.ResNetBack = ResNetBackbone(ResNetBlock, [3, 4, 6, 3], [64, 128, 256, 512])#resnet34

        self.pool = pool
        self.classification = classification
        self.in_channels = in_channels
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.patch_embedding1 = PatchEmbeddingBlock(
            in_channels=in_channels,
            img_size=img_size,
            patch_size=patch_size,
            hidden_size=hidden_size1,
            num_heads=num_heads,
            pos_embed=pos_embed,
            dropout_rate=dropout_rate,
            spatial_dims=spatial_dims,
        )
        self.blocks1 = nn.ModuleList(
            [TransformerBlock(hidden_size2, mlp_dim, num_heads, dropout_rate) for i in range(num_layers)]
        )
        self.norm1 = nn.LayerNorm(hidden_size2)
        if self.classification:
            self.cls_token1 = nn.Parameter(torch.zeros(1, 1, hidden_size2))
            if post_activation == "Tanh":
                self.classification_head = nn.Sequential(nn.Linear(hidden_size2, num_classes), nn.Tanh())
            else:
                self.classification_head = nn.Linear(hidden_size2, num_classes)  # type: ignore


    def forward(self, x1, x2):
        x1 = self.ResNetBack(x1)
        #x1 = x1.reshape(x1.shape[0],x1.shape[1],-1)# batchsize*512*flatten_size
        if self.in_channels == 1:
            x1 = x1.reshape(x1.shape[0],1,-1,x1.shape[3],x1.shape[4])
        x1 = self.patch_embedding1(x1)
        x1 = x1.reshape(x1.shape[0],-1,self.hidden_size2)
        x2 = x2.reshape(x2.shape[0],-1,self.hidden_size2)
        #print(x1.shape)
        #print(x2.shape)
        x1 = torch.cat((x1, x2), dim=1)
        
        if hasattr(self, "cls_token"):
            cls_token1 = self.cls_token1.expand(x1.shape[0], -1, -1)
            x1 = torch.cat((cls_token1, x1), dim=1)
            #cls_token2 = self.cls_token2.expand(x2.shape[0], -1, -1)
            #x2 = torch.cat((cls_token2, x2), dim=1)
        hidden_states_out1 = []
        for blk in self.blocks1:
            x1 = blk(x1)
            hidden_states_out1.append(x1)
        x1 = self.norm1(x1)

        
        if hasattr(self, "classification_head"):
            x1 = x1.mean(dim=1) if self.pool == 'mean' else x1[:, 0]
            #x2 = x2.mean(dim=1) if self.pool == 'mean' else x2[:, 0]
            #x = self.classification_head(torch.cat((x1, x2), 1))
            x1 = self.classification_head(x1)
        return x1, hidden_states_out1


#Parameters Setting

In [ ]:
key = '_HRLdemo'
rootpath = LLDdir
images_dir = rootpath+'/images'

modelname = 'HRL34';#HRL18,HRL34,HRL50
max_epochs = 300
pretrained = True
pretrained_model_path = rootpath+'/pretrained/'
trained = False
categories = 2
imagesize = (176, 208, 176)
usefreesurfer = False

downsampled = False
samplespace = 1
if downsampled:
    samplespace = 2 #2,4,8

best_model_names = []
savename='_CELoss'+'_downsample'+str(samplespace)+key
if trained:
    pretrained = False
    savedir = rootpath+'/models/LLD'+str(categories)+'_trained_'
elif pretrained:
    savedir = rootpath+'/models/LLD'+str(categories)+'_pretrained_'
else:
    savedir = rootpath+'/models/LLD'+str(categories)+'_scratch_'


pin_memory = torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#Dataset and Data loader

In [ ]:
class MDD_Dataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, augment=False):
        subjects = []            
        for (image_path,label) in zip(image_paths,labels):
            subject = tio.Subject(
                mri=tio.ScalarImage(image_path),
                labels=int(label),
            )
            subjects.append(subject)
        self.transform()
        if augment:
            self.dataset = tio.SubjectsDataset(subjects, transform=self.aug_transform)
        else:
            self.dataset = tio.SubjectsDataset(subjects, transform=self.preproc_transform)
            
    def transform(self):
        get_foreground = tio.ZNormalization.mean
        LI_LANDMARKS = "0 8.06305571158 15.5085721044 18.7007018006 21.5032879029 26.1413278906 29.9862059045 33.8384058795 38.1891334787 40.7217966068 44.0109152758 58.3906435207 100.0"
        LI_LANDMARKS = np.array([float(n) for n in LI_LANDMARKS.split()])
        landmarks_dict = {'mri': LI_LANDMARKS}
        preprocess = tio.Compose([
            tio.ToCanonical(),
            tio.Resample((samplespace, samplespace, samplespace)),                  # to MNI space (which is RAS+)
            #tio.CropOrPad((181, 220, 181)),                                              # tight crop around brain
            tio.HistogramStandardization(landmarks_dict, masking_method=get_foreground),  # standard histogram
            tio.ZNormalization(masking_method=get_foreground),                            # zero mean and unit std
            tio.RescaleIntensity(percentiles=(0.5,99.5), out_min_max=(0, 1), in_min_max=(-5, 5))
        ])
        augment = tio.Compose([
            tio.RandomAffine(scales=0.1,degrees=10,translation=5,isotropic=True,center='image'),       # random affine
            #tio.RandomAffine(scales=0.2,degrees=20,translation=10,isotropic=True,center='image'),       # random affine
        ])

        self.aug_transform = tio.Compose([preprocess, augment])
        self.preproc_transform = preprocess

def get_loader(imagepaths,label, batch_size = 1, augment=False):
    dataset = MDD_Dataset(image_paths=imagepaths, labels=label, augment=augment)
    if augment:
        loader = DataLoader(dataset.dataset,batch_size=batch_size,num_workers=batch_size,shuffle=False,pin_memory=pin_memory,drop_last=False)
    else:
        loader = DataLoader(dataset.dataset,batch_size=batch_size,pin_memory=pin_memory)
    return loader


#Pretrained Models

In [ ]:
Resnet_model_names = []
if pretrained:
    if modelname == 'HRL18' and len(Resnet_model_names) == 0:
        Resnet_model_names.append(rootpath+'/pretrained/resnet_18_23dataset.pth')
    if modelname == 'HRL34' and len(Resnet_model_names) == 0:
        Resnet_model_names.append(rootpath+'/pretrained/resnet_34_23dataset.pth')
    if modelname == 'HRL50' and len(Resnet_model_names) == 0:
        Resnet_model_names.append(rootpath+'/pretrained/resnet_50_23dataset.pth')


# Model train

In [ ]:
kfold = 5
fold = 0
batchsize = 2
modelname2 = key +'_head16hidden128dorp0.0'
features_dict = np.load(rootpath+'/images/Demo_LLD'+str(categories)+'category_features_dict.npy',allow_pickle=True).item()
IDs = np.array(features_dict['demo_IDs'])
All_labels = np.array(features_dict['demo_labels'])
Freesurfer_features = features_dict['demo_features']
for idx in range(kfold):
    fold = fold+1
    filename1=rootpath+'/files/'+str(categories)+'category_Fold'+str(fold)+'_train_list.txt'
    labels0  = np.genfromtxt(filename1,dtype=str)
    image_paths = labels0[...,0].tolist()

    train_image_paths = []
    index = []
    for imagepath in image_paths:
        train_ID = re.findall(r'\d\d\d',imagepath)[0]
        idx1 = np.where(str(train_ID) == IDs)[0]
        if idx1.size>0:
            train_image_paths.append(rootpath+'/'+imagepath)
            index.append(idx1[0])
    index = np.array(index)
    train_features = Freesurfer_features[index]
    train_features = normalize(train_features, axis=0, norm='max')

    train_labels = All_labels[index].tolist()

    train_loader = get_loader(train_image_paths,train_labels, batch_size = batchsize, augment=True)

    model = HRL34_v2(in_channels=512, img_size=(3,4,3), patch_size=(3,4,3),num_heads=16,hidden_size1=128,hidden_size2=128,dropout_rate= 0.0, num_layers=1, pos_embed='conv', classification=True, num_classes=categories,spatial_dims=3)

    if pretrained:
        path = Resnet_model_names[0]
        save_model = torch.load(path)
        save_dict = save_model.items()

        model_dict = model.state_dict()
        resnet_state_dict = {'ResNetBack.'+k:v for k,v in save_model.items() if 'ResNetBack.'+k in model_dict.keys()}
        model.load_state_dict(resnet_state_dict, strict = False)
        #for name, value in model_ResViT.named_parameters():
            #if name.startswith('ResNetBack'):
                #value.requires_grad = False

    model.to(device)    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_function = torch.nn.CrossEntropyLoss()

    min_v_loss = np.inf
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = []
    metric_values = []

    threshhold = 0.79 
    for epoch in range(max_epochs):
        print("-" * 10)
        print(f"Fold{fold} epoch {epoch + 1}/{max_epochs}")
        epoch_loss = 0
        step = 0
        train_pred  = []
        train_prob  = []
        train_label = []
        model.train()
        for batch_data in tqdm(train_loader):
            inputs1 = batch_data['mri'][tio.DATA].to(torch.float32).to(device)
            labels = batch_data['labels'].to(device)

            #inputs2 = train_features[step*batchsize:(step+1)*batchsize,0:1664]
            inputs2 = train_features[step*batchsize:(step+1)*batchsize,0:896]
            inputs2 = torch.from_numpy(inputs2).to(torch.float32).to(device)
            
            step += 1

            optimizer.zero_grad()
            outputs = model(inputs1,inputs2)
            loss = loss_function(outputs[0], labels)
            loss.backward()
            optimizer.step()
                
            train_pred.append(F.softmax(outputs[0], dim=1).max(dim=1)[1])
            train_prob.append(F.softmax(outputs[0], dim=1))
            train_label.append(labels)

            epoch_loss += loss.item()
            epoch_len = len(train_image_paths) // train_loader.batch_size
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        y_prob = torch.cat(train_prob, dim=0).cpu().detach().numpy()
        y_pred = torch.cat(train_pred, dim=0).cpu().detach().numpy()
        y_true = torch.cat(train_label, dim=0).cpu().detach().numpy()

        epoch_report = classification_report(y_true, y_pred, output_dict = True,target_names=['CN_N', 'CN_D'],zero_division=0)
        metric = epoch_report['accuracy']

        if (metric > best_metric) and (metric > threshhold):
            threshhold = min(threshhold+0.05,0.95)
            best_metric = metric
            best_metric_epoch = epoch + 1
            best_model_name = savedir+modelname2+savename+'_KFold%02i'% (fold)+'_epoch%02i_acc%.2f.pth' % (best_metric_epoch,best_metric)
            torch.save(model.state_dict(), best_model_name)
            print("saved new best metric model")
        if (metric > 0.95):
            break
        print(f"Current epoch: {epoch+1} current accuracy: {metric:.4f} ")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")

    best_model_names.append(best_model_name)
    print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
print('Training finish!')


----------
Fold1 epoch 1/300


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 1 average loss: 0.6952
Current epoch: 1 current accuracy: 0.7500 
Best accuracy: -1.0000 at epoch -1
----------
Fold1 epoch 2/300


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 2 average loss: 0.3085
saved new best metric model
Training completed, best_metric: 1.0000 at epoch: 2
----------
Fold2 epoch 1/300


...

epoch 1 average loss: 0.8318
Current epoch: 1 current accuracy: 0.7500 
Best accuracy: -1.0000 at epoch -1
----------
Fold2 epoch 2/300


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 2 average loss: 0.6165
Current epoch: 2 current accuracy: 0.7500 
Best accuracy: -1.0000 at epoch -1
----------
Fold2 epoch 3/300


  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Model test

trained models

In [ ]:
trained_model_names = []
if pretrained:
    if modelname == 'HRL34' and len(trained_model_names) == 0:
        trained_model_names.append(rootpath+'/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold01.pth')
        trained_model_names.append(rootpath+'/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold02.pth')
        trained_model_names.append(rootpath+'/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold03.pth')
        trained_model_names.append(rootpath+'/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold04.pth')
        trained_model_names.append(rootpath+'/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold05.pth')


In [ ]:
print('Testing start!')
test_CN_N = []
test_CN_D = []
test_CI = []

test_AUC =[]
test_ACC =[]
test_PRE =[]
test_SEN =[]
test_SPE =[]
test_F1s =[]
test_MCC =[]
test_report = []
test_confus_mtrx = []

features_dict = np.load(rootpath+'/images/Demo_LLD'+str(categories)+'category_features_dict.npy',allow_pickle=True).item()
IDs = np.array(features_dict['demo_IDs'])
All_labels = np.array(features_dict['demo_labels'])
Freesurfer_features = features_dict['demo_features']

kfold = 5
fold = 0
batchsize = 1
for idx in range(kfold):
    fold+=1
    filename1=rootpath+'/files/'+str(categories)+'category_Fold'+str(fold)+'_test_list.txt'
    labels0  = np.genfromtxt(filename1,dtype=str)
    image_paths = labels0[...,0].tolist()
    test_image_paths = []
    index = []

    for imagepath in image_paths:
        test_ID = re.findall(r'\d\d\d',imagepath)[0]
        idx1 = np.where(str(test_ID) == IDs)[0]
        if idx1.size>0:
            test_image_paths.append(rootpath+'/'+imagepath)
            index.append(idx1[0])
    test_features = Freesurfer_features[np.array(index)]
    test_features = normalize(test_features, axis=0, norm='max')
    test_labels   = All_labels[np.array(index)].tolist()

    loader = get_loader(test_image_paths,test_labels, batch_size = batchsize)


    trained_model = trained_model_names[idx]
    print(trained_model)
    model.load_state_dict(torch.load(trained_model), strict = False)
       
    model.to(device)
    min_v_loss = np.inf
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = []
    metric_values = []

    print("-" * 10)
    print(f"Fold{fold}")
    model.eval()
    
    pred = []
    prob = []
    label = []
    predict= []
    step  = 0
    num_correct = 0.0
    metric_count = 0
    for test_data in tqdm(loader):
        inputs1 = test_data['mri'][tio.DATA].to(torch.float32).to(device)
        test_labels = test_data['labels']
        inputs2 = test_features[step*batchsize:(step+1)*batchsize,0:896]
        inputs2 = torch.from_numpy(inputs2).to(torch.float32).to(device)
        
        step += 1

        with torch.no_grad():
            test_outputs = model(inputs1, inputs2)

        outprob = F.softmax(test_outputs[0], dim=1)
        prob.append(outprob)
        pred.append(outprob.max(dim=1)[1])
        predict.append(torch.where(outprob>0.5,torch.ones_like(outprob),torch.zeros_like(outprob)))
        label.append(test_labels)

    y_prob = torch.cat(prob, dim=0).cpu().detach().numpy()
    y_true = torch.cat(label, dim=0).cpu().detach().numpy()
    y_pred =  np.array(torch.cat(predict, dim=0).cpu().detach().numpy()[...,1],dtype=int)
    
    AUC = roc_auc_score(y_true,y_pred) 
    MCC = matthews_corrcoef(y_true, y_pred)

    confus_mtrx = confusion_matrix(y_true, y_pred).ravel()
    #SPE  = confus_mtrx[0]/(confus_mtrx[0]+confus_mtrx[2])
    epoch_report = classification_report(y_true, y_pred, output_dict = True,target_names=['CN_N', 'CN_D'],zero_division=0)
    SPE = epoch_report['CN_N']['recall']
    PRE = epoch_report['CN_D']['precision']
    SEN = epoch_report['CN_D']['recall']
    F1s = epoch_report['CN_D']['f1-score']
    
    test_AUC.append(AUC)
    test_ACC.append(epoch_report['accuracy'])
    test_PRE.append(PRE)
    test_SEN.append(SEN)
    test_SPE.append(SPE)
    test_F1s.append(F1s)
    test_MCC.append(MCC)
    
    print('%02i Fold testing finish!'% (fold))
 
mean_AUC =round(np.mean(test_AUC)*100,2)
mean_ACC =round(np.mean(test_ACC)*100,2)
mean_PRE =round(np.mean(test_PRE)*100,2)
mean_SEN =round(np.mean(test_SEN)*100,2)
mean_SPE =round(np.mean(test_SPE)*100,2)
mean_F1s =round(np.mean(test_F1s)*100,2)
mean_MCC =round(np.mean(test_MCC)*100,2)

std_AUC =round(np.std(test_AUC,ddof=1)*100,2)
std_ACC =round(np.std(test_ACC,ddof=1)*100,2)
std_PRE =round(np.std(test_PRE,ddof=1)*100,2)
std_SEN =round(np.std(test_SEN,ddof=1)*100,2)
std_SPE =round(np.std(test_SPE,ddof=1)*100,2)
std_F1s =round(np.std(test_F1s,ddof=1)*100,2)
std_MCC =round(np.std(test_MCC,ddof=1)*100,2)

print(f'AUC:{mean_AUC}\xB1{std_AUC}, ACC:{mean_ACC}\xB1{std_ACC}, PRE:{mean_PRE}\xB1{std_PRE}, SEN:{mean_SEN}\xB1{std_SEN}, SPE:{mean_SPE}\xB1{std_SPE}, F1S:{mean_F1s}\xB1{std_F1s}, MCC:{mean_MCC}\xB1{std_MCC}')
print('Testing finish!')

Testing start!
/content/drive/MyDrive/Colab Notebooks/LLD_Progression/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold01.pth
----------
Fold1


  0%|          | 0/4 [00:00<?, ?it/s]

01 Fold testing finish!
/content/drive/MyDrive/Colab Notebooks/LLD_Progression/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold02.pth
----------
Fold2


  0%|          | 0/4 [00:00<?, ?it/s]

02 Fold testing finish!
/content/drive/MyDrive/Colab Notebooks/LLD_Progression/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold03.pth
----------
Fold3


...

03 Fold testing finish!
/content/drive/MyDrive/Colab Notebooks/LLD_Progression/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold04.pth
----------
Fold4


  0%|          | 0/4 [00:00<?, ?it/s]

04 Fold testing finish!
/content/drive/MyDrive/Colab Notebooks/LLD_Progression/models/LLD2_scratch_HRLv2_head16hidden128dorp0.0_CELoss_downsample1_KFold05.pth
----------
Fold5


  0%|          | 0/4 [00:00<?, ?it/s]

05 Fold testing finish!
AUC:50.0±0.0, ACC:40.0±13.69, PRE:40.0±13.69, SEN:100.0±0.0, SPE:0.0±0.0, F1S:56.0±14.61, MCC:0.0±0.0
Testing finish!
